The Seasonal Autoregressive Integrated Moving Average, or SARIMA, model is an approach for modeling univariate time series data that may contain trend and seasonal components. It is an effective approach for time series forecasting, although it requires careful analysis and domain expertise in order to configure the seven or more model hyperparameters. 

- How to develop a framework for grid searching SARIMA models from scratch using walk-forward validation.
- How to grid search SARIMA model hyperparameters for daily time series data for births.
- How to grid search SARIMA model hyperparameters for monthly time series data for shampoo sales, car sales, and temperature.

## 13.1 Tutorial Overview
This tutorial is divided into five parts; they are: 
1. Develop a Grid Search Framework
2. Case Study 1: No Trend or Seasonality
3. Case Study 2: Trend
4. Case Study 3: Seasonality
5. Case Study 4: Trend and Seasonality

## 13.2 Develop a Grid Search Framework

This model has hyperparameters that control the nature of the model performed for the series, trend and seasonality, specifically:
- order: A tuple p, d, and q parameters for the modeling of the trend.
- seasonal order: A tuple of P, D, Q, and m parameters for the modeling the seasonality
- trend: A parameter for controlling a model of the deterministic trend as one of ‘n’, ‘c’, ‘t’, and ‘ct’ for no trend, constant, linear, and constant with linear trend, respectively.


1. We can start-off by defining a function that will fit a model with a given configuration and make a one-step forecast. The sarima forecast() below implements this behavior.
2. We can start-off by defining a function that will fit a model with a given configuration and make a one-step forecast. The sarima forecast() below implements this behavior.
3. The only parameter we may want to specify is the periodicity of the seasonal component in the series, if one exists. By default, we will assume no seasonal component. The sarima configs() function below will create a list of model configurations to evaluate.

In [1]:
# grid search sarima hyperparameters
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = []
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = []
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

if __name__ == '__main__':
	# define dataset
	data = [10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]
	print(data)
	# data split
	n_test = 4
	# model configs
	cfg_list = sarima_configs()
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

[10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 85.732
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'n']] 6.103
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'n']] 0.000
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'n']] 3.129
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'n']] 85.732
 > Model[[(0, 0, 0), (2, 0, 1, 0), 'n']] 0.000
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'c']] 37.914
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'ct']] 81.843
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'c']] 42.866
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'c']] 0.001
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'ct']] 0.000
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'n']] 85.732
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'ct']] 10.000
 > Model[[(0, 0, 1), (0, 0, 1, 0), 'n']] 85.732
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'ct']] 60.381
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'ct']] 75.829
 > Model[[(0, 0, 1), (1, 0, 0, 0), 'n']] 6.104
 > Model[[(0, 0, 1), (2, 0, 0, 0), 'n']] 0.000
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'c']] 0.002
 > Model[[(0, 0, 1), (1, 0, 1,

 > Model[[(2, 1, 1), (2, 0, 0, 0), 'n']] 0.000
 > Model[[(2, 0, 0), (1, 0, 0, 0), 'c']] 0.000
 > Model[[(2, 0, 0), (0, 0, 0, 0), 'ct']] 0.000
 > Model[[(2, 1, 1), (2, 0, 0, 0), 'c']] 0.045
 > Model[[(2, 0, 0), (0, 0, 1, 0), 'ct']] 0.096
 > Model[[(2, 1, 1), (2, 0, 1, 0), 'c']] 0.033
 > Model[[(2, 1, 1), (0, 0, 0, 0), 't']] 0.000
 > Model[[(2, 1, 1), (0, 0, 1, 0), 't']] 0.000
 > Model[[(2, 1, 1), (1, 0, 0, 0), 't']] 0.000
 > Model[[(2, 1, 1), (1, 0, 1, 0), 't']] 0.000
 > Model[[(2, 1, 1), (2, 0, 0, 0), 't']] 0.000
 > Model[[(2, 1, 1), (2, 0, 1, 0), 't']] 0.000
 > Model[[(2, 0, 0), (1, 0, 1, 0), 'c']] 0.000
 > Model[[(2, 0, 0), (1, 0, 0, 0), 'ct']] 0.000
 > Model[[(2, 1, 1), (2, 0, 0, 0), 'ct']] 0.035
 > Model[[(2, 1, 1), (2, 0, 1, 0), 'ct']] 0.200
 > Model[[(2, 0, 0), (2, 0, 0, 0), 'c']] 0.000
 > Model[[(2, 0, 0), (1, 0, 1, 0), 'ct']] 0.000
 > Model[[(2, 0, 0), (2, 0, 1, 0), 'c']] 0.000
 > Model[[(2, 0, 0), (2, 0, 0, 0), 'ct']] 0.000
 > Model[[(2, 0, 0), (0, 0, 0, 0), 't']] 0.001
 > Mod

Running the example first prints the contrived time series dataset. Next, the model configurations and their errors are reported as they are evaluated, truncated below for brevity. Finally, the configurations and the error for the top three configurations are reported.

## 13.3 Case Study 1: No Trend or Seasonality

The daily female births dataset summarizes the daily total female births in California, USA in 1959. The dataset has one year, or 365 observations. We will use the first 200 for training and the remaining 165 as the test set.

In [3]:
# grid search sarima hyperparameters for daily female dataset
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_csv

# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

if __name__ == '__main__':
	# load dataset
	series = read_csv('daily-total-female-births.csv', header=0, index_col=0)
	data = series.values
	# data split
	n_test = 165
	# model configs
	cfg_list = sarima_configs()
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 44.930
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'n']] 8.708
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'n']] 26.223
 > Model[[(0, 0, 0), (0, 0, 2, 0), 'n']] 19.048
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'n']] 7.114
 > Model[[(0, 0, 0), (2, 0, 0, 0), 't']] 7.868
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'c']] 7.299
 > Model[[(0, 0, 0), (1, 0, 2, 0), 'n']] 7.046
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'n']] 7.893
 > Model[[(0, 0, 0), (2, 0, 2, 0), 'ct']] 7.000
 > Model[[(0, 0, 0), (2, 0, 1, 0), 'n']] 7.019
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'n']] 26.223
 > Model[[(0, 0, 0), (2, 0, 1, 0), 'c']] 7.354
 > Model[[(0, 0, 1), (0, 0, 1, 0), 'n']] 21.764
 > Model[[(0, 0, 0), (2, 0, 1, 0), 't']] 7.212
 > Model[[(0, 0, 1), (0, 0, 2, 0), 'n']] 16.280
 > Model[[(0, 0, 0), (2, 0, 2, 0), 'n']] 7.083
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'c']] 7.718
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'c']] 7.450
 > Model[[(0, 0, 1), (1, 0, 0, 0), 'n']] 7.447
 > Model[[(0, 0, 0), (2, 0, 2, 0), 'c']] 7.216
 > Mod

 > Model[[(0, 1, 1), (1, 0, 0, 0), 'ct']] 7.059
 > Model[[(0, 1, 1), (1, 0, 2, 0), 'ct']] 6.987
 > Model[[(0, 1, 1), (1, 0, 1, 0), 'ct']] 7.065
 > Model[[(0, 1, 2), (0, 0, 0, 0), 'n']] 7.014
 > Model[[(0, 1, 2), (0, 0, 1, 0), 'n']] 7.019
 > Model[[(0, 1, 1), (2, 0, 0, 0), 'ct']] 7.087
 > Model[[(0, 1, 2), (0, 0, 2, 0), 'n']] 7.111
 > Model[[(0, 1, 1), (2, 0, 1, 0), 'ct']] 6.968
 > Model[[(0, 1, 2), (1, 0, 0, 0), 'n']] 7.004
 > Model[[(0, 1, 2), (1, 0, 1, 0), 'n']] 7.014
 > Model[[(0, 1, 2), (2, 0, 0, 0), 'n']] 7.018
 > Model[[(0, 1, 2), (1, 0, 2, 0), 'n']] 6.999
 > Model[[(0, 1, 1), (2, 0, 2, 0), 'ct']] 6.986
 > Model[[(0, 1, 2), (0, 0, 0, 0), 'c']] 7.042
 > Model[[(0, 1, 2), (2, 0, 1, 0), 'n']] 7.016
 > Model[[(0, 1, 2), (0, 0, 1, 0), 'c']] 7.036
 > Model[[(0, 1, 2), (2, 0, 2, 0), 'n']] 6.996
 > Model[[(0, 1, 2), (1, 0, 0, 0), 'c']] 7.021
 > Model[[(0, 1, 2), (0, 0, 2, 0), 'c']] 7.055
 > Model[[(0, 1, 2), (1, 0, 1, 0), 'c']] 7.025
 > Model[[(0, 1, 2), (1, 0, 2, 0), 'c']] 7.031
 > Mode

 > Model[[(1, 1, 0), (0, 0, 0, 0), 'ct']] 7.946
 > Model[[(1, 1, 0), (1, 0, 2, 0), 't']] 7.052
 > Model[[(1, 1, 0), (2, 0, 1, 0), 't']] 7.127
 > Model[[(1, 1, 0), (1, 0, 0, 0), 'ct']] 7.811
 > Model[[(1, 1, 0), (0, 0, 1, 0), 'ct']] 7.076
 > Model[[(1, 1, 0), (2, 0, 2, 0), 't']] 7.096
 > Model[[(1, 1, 0), (0, 0, 2, 0), 'ct']] 7.073
 > Model[[(1, 1, 0), (2, 0, 0, 0), 'ct']] 7.356
 > Model[[(1, 1, 0), (1, 0, 1, 0), 'ct']] 7.089
 > Model[[(1, 1, 1), (0, 0, 0, 0), 'n']] 7.001
 > Model[[(1, 1, 0), (1, 0, 2, 0), 'ct']] 7.123
 > Model[[(1, 1, 1), (0, 0, 1, 0), 'n']] 7.004
 > Model[[(1, 1, 1), (1, 0, 0, 0), 'n']] 7.004
 > Model[[(1, 1, 0), (2, 0, 1, 0), 'ct']] 7.044
 > Model[[(1, 1, 1), (0, 0, 2, 0), 'n']] 7.014
 > Model[[(1, 1, 1), (1, 0, 1, 0), 'n']] 7.047
 > Model[[(1, 1, 1), (2, 0, 0, 0), 'n']] 7.014
 > Model[[(1, 1, 0), (2, 0, 2, 0), 'ct']] 7.013
 > Model[[(1, 1, 1), (1, 0, 2, 0), 'n']] 6.983
 > Model[[(1, 1, 1), (2, 0, 1, 0), 'n']] 7.020
 > Model[[(1, 1, 1), (0, 0, 0, 0), 'c']] 7.037
 > M

 > Model[[(2, 0, 2), (2, 0, 2, 0), 'c']] 7.234
 > Model[[(2, 0, 2), (0, 0, 1, 0), 't']] 7.122
 > Model[[(2, 0, 2), (0, 0, 2, 0), 't']] 7.107
 > Model[[(2, 0, 2), (1, 0, 0, 0), 't']] 7.229
 > Model[[(2, 0, 2), (1, 0, 1, 0), 't']] 7.146
 > Model[[(2, 0, 2), (1, 0, 2, 0), 't']] 7.107
 > Model[[(2, 0, 2), (2, 0, 0, 0), 't']] 7.089
 > Model[[(2, 0, 2), (0, 0, 0, 0), 'ct']] 7.044
 > Model[[(2, 0, 2), (2, 0, 1, 0), 't']] 7.069
 > Model[[(2, 0, 2), (2, 0, 2, 0), 't']] 7.273
 > Model[[(2, 0, 2), (0, 0, 1, 0), 'ct']] 7.038
 > Model[[(2, 0, 2), (1, 0, 0, 0), 'ct']] 7.030
 > Model[[(2, 0, 2), (0, 0, 2, 0), 'ct']] 7.057
 > Model[[(2, 0, 2), (1, 0, 1, 0), 'ct']] 7.060
 > Model[[(2, 0, 2), (1, 0, 2, 0), 'ct']] 7.073
 > Model[[(2, 1, 0), (0, 0, 0, 0), 'n']] 7.519
 > Model[[(2, 0, 2), (2, 0, 0, 0), 'ct']] 7.055
 > Model[[(2, 1, 0), (0, 0, 1, 0), 'n']] 6.993
 > Model[[(2, 1, 0), (1, 0, 0, 0), 'n']] 7.299
 > Model[[(2, 0, 2), (2, 0, 1, 0), 'ct']] 7.049
 > Model[[(2, 1, 0), (0, 0, 2, 0), 'n']] 7.023
 > Mo

We can see that the best result was an RMSE of about 6.64 births. A naive model achieved an RMSE of 6.93 births suggesting that the best performing SARIMA model is skillful on this problem. We can unpack the configuration of the best performing model as follows:
- Order: (1, 0, 2)
- Seasonal Order: (1, 0, 1, 0)
- Trend Parameter: ‘t’ for linear trend

It is surprising that a configuration with some seasonal elements resulted in the lowest error. I would not have guessed at this configuration and would have likely stuck with an ARIMA model.

## 13.4 Case Study 2: Trend

- The monthly shampoo sales dataset summarizes the monthly sales of shampoo over a three-year period. 
- The dataset has three years, or 36 observations. We will use the first 24 for training and the remaining 12 as the test set. 

In [4]:
# grid search sarima hyperparameters for monthly shampoo sales dataset
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_csv

# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = []
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = []
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

if __name__ == '__main__':
	# load dataset
	series = read_csv('monthly-shampoo-sales.csv', header=0, index_col=0)
	data = series.values
	# data split
	n_test = 12
	# model configs
	cfg_list = sarima_configs()
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 491.532
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'n']] 144.299
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'n']] 328.076
 > Model[[(0, 0, 0), (0, 0, 2, 0), 'n']] 264.182
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'n']] 84.467
 > Model[[(0, 0, 0), (0, 0, 2, 0), 't']] 76.739
 > Model[[(0, 0, 0), (1, 0, 2, 0), 'n']] 75.366
 > Model[[(0, 0, 0), (1, 0, 0, 0), 't']] 98.842
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'n']] 88.757
 > Model[[(0, 0, 0), (2, 0, 1, 0), 'n']] 72.974
 > Model[[(0, 0, 1), (1, 0, 2, 0), 'c']] 167.056
 > Model[[(0, 0, 0), (2, 0, 2, 0), 'ct']] 98.578
 > Model[[(0, 0, 0), (1, 0, 1, 0), 't']] 80.533
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'n']] 328.076
 > Model[[(0, 0, 1), (0, 0, 1, 0), 'n']] 295.520
 > Model[[(0, 0, 0), (2, 0, 2, 0), 'n']] 74.032
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'c']] 235.880
 > Model[[(0, 0, 1), (2, 0, 0, 0), 'c']] 66.579
 > Model[[(0, 0, 1), (0, 0, 2, 0), 'n']] 221.799
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'c']] 198.046
 > Model[[(0, 0, 0), (1, 0, 2

 > Model[[(0, 1, 1), (2, 0, 2, 0), 't']] 61.060
 > Model[[(0, 1, 1), (1, 0, 0, 0), 'ct']] 61.644
 > Model[[(0, 1, 1), (2, 0, 0, 0), 'ct']] 61.685
 > Model[[(0, 1, 1), (0, 0, 0, 0), 'ct']] 84.085
 > Model[[(0, 1, 1), (2, 0, 1, 0), 'c']] 80.448
 > Model[[(0, 1, 1), (1, 0, 1, 0), 'ct']] 65.941
 > Model[[(0, 1, 1), (0, 0, 1, 0), 'ct']] 61.852
 > Model[[(0, 1, 1), (2, 0, 1, 0), 'ct']] 62.368
 > Model[[(0, 1, 2), (0, 0, 0, 0), 'n']] 67.398
 > Model[[(0, 1, 2), (0, 0, 1, 0), 'n']] 69.639
 > Model[[(0, 1, 2), (0, 0, 2, 0), 'n']] 86.568
 > Model[[(0, 1, 1), (2, 0, 2, 0), 'c']] 86.099
 > Model[[(0, 1, 1), (1, 0, 2, 0), 'ct']] 69.426
 > Model[[(0, 1, 2), (1, 0, 0, 0), 'n']] 68.702
 > Model[[(0, 1, 2), (2, 0, 0, 0), 'n']] 73.184
 > Model[[(0, 1, 1), (2, 0, 2, 0), 'ct']] 71.727
 > Model[[(0, 1, 2), (1, 0, 2, 0), 'n']] 75.967
 > Model[[(0, 1, 2), (1, 0, 1, 0), 'n']] 70.734
 > Model[[(0, 1, 2), (0, 0, 0, 0), 'c']] 65.844
 > Model[[(0, 1, 2), (2, 0, 1, 0), 'n']] 69.778
 > Model[[(0, 1, 2), (2, 0, 2, 0

 > Model[[(1, 1, 0), (0, 0, 0, 0), 't']] 85.288
 > Model[[(1, 1, 0), (1, 0, 2, 0), 'c']] 75.560
 > Model[[(1, 1, 0), (0, 0, 1, 0), 't']] 64.100
 > Model[[(1, 1, 0), (2, 0, 1, 0), 'c']] 88.277
 > Model[[(1, 1, 0), (1, 0, 0, 0), 't']] 72.927
 > Model[[(1, 1, 0), (0, 0, 2, 0), 't']] 67.802
 > Model[[(1, 1, 0), (2, 0, 2, 0), 'c']] 85.733
 > Model[[(1, 1, 0), (1, 0, 1, 0), 't']] 65.254
 > Model[[(1, 1, 0), (2, 0, 0, 0), 't']] 76.902
 > Model[[(1, 1, 0), (0, 0, 0, 0), 'ct']] 86.514
 > Model[[(1, 1, 0), (1, 0, 2, 0), 't']] 65.582
 > Model[[(1, 1, 0), (2, 0, 1, 0), 't']] 69.888
 > Model[[(1, 1, 0), (0, 0, 1, 0), 'ct']] 61.644
 > Model[[(1, 1, 0), (2, 0, 2, 0), 't']] 69.508
 > Model[[(1, 1, 0), (0, 0, 2, 0), 'ct']] 69.171
 > Model[[(1, 1, 0), (1, 0, 0, 0), 'ct']] 74.367
 > Model[[(1, 1, 0), (1, 0, 1, 0), 'ct']] 61.853
 > Model[[(1, 1, 0), (2, 0, 0, 0), 'ct']] 78.481
 > Model[[(1, 1, 1), (0, 0, 0, 0), 'n']] 91.909
 > Model[[(1, 1, 0), (1, 0, 2, 0), 'ct']] 61.242
 > Model[[(1, 1, 1), (0, 0, 1, 0)

 > Model[[(2, 0, 2), (2, 0, 0, 0), 'n']] 83.103
 > Model[[(2, 0, 2), (1, 0, 2, 0), 'n']] 74.864
 > Model[[(2, 0, 2), (2, 0, 1, 0), 'n']] 72.719
 > Model[[(2, 0, 2), (2, 0, 2, 0), 'n']] 74.830
 > Model[[(2, 0, 2), (0, 0, 0, 0), 'c']] 73.020
 > Model[[(2, 0, 2), (0, 0, 1, 0), 'c']] 89.445
 > Model[[(2, 0, 2), (0, 0, 2, 0), 'c']] 103.717
 > Model[[(2, 0, 2), (1, 0, 0, 0), 'c']] 76.490
 > Model[[(2, 0, 2), (1, 0, 1, 0), 'c']] 76.876
 > Model[[(2, 0, 2), (2, 0, 0, 0), 'c']] 82.139
 > Model[[(2, 0, 2), (1, 0, 2, 0), 'c']] 88.158
 > Model[[(2, 0, 2), (2, 0, 1, 0), 'c']] 81.623
 > Model[[(2, 0, 2), (0, 0, 0, 0), 't']] 65.983
 > Model[[(2, 0, 2), (0, 0, 1, 0), 't']] 67.256
 > Model[[(2, 0, 2), (2, 0, 2, 0), 'c']] 81.600
 > Model[[(2, 0, 2), (0, 0, 2, 0), 't']] 67.234
 > Model[[(2, 0, 2), (1, 0, 0, 0), 't']] 64.310
 > Model[[(2, 0, 2), (1, 0, 1, 0), 't']] 66.250
 > Model[[(2, 0, 2), (1, 0, 2, 0), 't']] 70.545
 > Model[[(2, 0, 2), (2, 0, 0, 0), 't']] 69.498
 > Model[[(2, 0, 2), (2, 0, 1, 0), 't']

We can see that the best result was an RMSE of about 54.28 sales. A naive model achieved an RMSE of 95.69 sales on this dataset, meaning that the best performing SARIMA model is skillful on this problem. We can unpack the configuration of the best performing model as follows:
- Trend Order: (0, 1, 2)
- Seasonal Order: (2, 0, 2, 0)
- Trend Parameter: ‘t’ (linear trend)

## 13.5 Case Study 3: Seasonality

- The monthly mean temperatures dataset summarizes the monthly average air temperatures in Nottingham Castle, England from 1920 to 1939 in degrees Fahrenheit. 
- The dataset has 20 years, or 240 observations. We will trim the dataset to the last five years of data (60 observations) in order to speed up the model evaluation process and use the last year or 12 observations for the test set.

In [5]:
# grid search sarima hyperparameters for monthly mean temp dataset
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_csv

# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

if __name__ == '__main__':
	# load dataset
	series = read_csv('monthly-mean-temp.csv', header=0, index_col=0)
	data = series.values
	# trim dataset to 5 years
	data = data[-(5*12):]
	# data split
	n_test = 12
	# model configs
	cfg_list = sarima_configs(seasonal=[0, 12])
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

 > Model[[(0, 0, 0), (0, 0, 0, 12), 'n']] 50.075
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 50.075
 > Model[[(0, 0, 0), (0, 1, 0, 12), 'n']] 2.195
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'n']] 5.182
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'n']] 26.003
 > Model[[(0, 0, 0), (1, 0, 0, 12), 'n']] 2.166
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'n']] 5.001
 > Model[[(0, 0, 0), (0, 0, 1, 12), 'n']] 32.867
 > Model[[(0, 0, 0), (0, 0, 2, 0), 'n']] 20.952
 > Model[[(0, 0, 0), (1, 1, 0, 12), 'n']] 1.647
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'n']] 4.895
 > Model[[(0, 0, 0), (1, 0, 2, 0), 'n']] 4.562
 > Model[[(0, 0, 0), (2, 0, 0, 12), 'n']] 1.731
 > Model[[(0, 0, 0), (1, 0, 1, 12), 'n']] 1.519
 > Model[[(0, 0, 0), (2, 0, 1, 0), 'n']] 4.851
 > Model[[(0, 0, 0), (2, 1, 0, 12), 'n']] 2.138
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'c']] 8.325
 > Model[[(0, 0, 0), (0, 0, 0, 12), 'c']] 8.325
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'c']] 5.779
 > Model[[(0, 0, 0), (2, 0, 2, 0), 'n']] 4.507
 > Model[[(0, 0, 0), (0, 1, 0, 12), 'c']] 2.25

 > Model[[(0, 0, 2), (1, 1, 0, 12), 'c']] 2.238
 > Model[[(0, 0, 2), (1, 0, 2, 0), 'c']] 4.111
 > Model[[(0, 0, 2), (2, 0, 0, 0), 'c']] 4.982
 > Model[[(0, 0, 2), (1, 0, 1, 12), 'c']] 3.474
 > Model[[(0, 0, 2), (2, 0, 1, 0), 'c']] 3.802
 > Model[[(0, 0, 2), (2, 0, 0, 12), 'c']] 2.038
 > Model[[(0, 0, 2), (0, 0, 0, 0), 't']] 11.286
 > Model[[(0, 0, 2), (2, 0, 2, 0), 'c']] 3.532
 > Model[[(0, 0, 2), (0, 0, 0, 12), 't']] 11.286
 > Model[[(0, 0, 2), (0, 0, 1, 0), 't']] 8.076
 > Model[[(0, 0, 2), (2, 1, 0, 12), 'c']] 3.294
 > Model[[(0, 0, 2), (2, 0, 1, 12), 'c']] 7.760
 > Model[[(0, 0, 2), (0, 1, 0, 12), 't']] 2.552
 > Model[[(0, 0, 2), (0, 0, 2, 0), 't']] 6.772
 > Model[[(0, 0, 2), (0, 0, 1, 12), 't']] 6396.344
 > Model[[(0, 0, 2), (1, 0, 0, 0), 't']] 4.717
 > Model[[(0, 0, 2), (1, 0, 0, 12), 't']] 3.511
 > Model[[(0, 0, 2), (1, 0, 1, 0), 't']] 5.199
 > Model[[(0, 0, 2), (1, 0, 1, 12), 't']] 9375.568
 > Model[[(0, 0, 2), (1, 1, 0, 12), 't']] 2.274
 > Model[[(0, 0, 2), (1, 0, 2, 0), 't']] 

 > Model[[(0, 1, 1), (2, 0, 1, 12), 't']] 2.586
 > Model[[(0, 1, 1), (2, 1, 0, 12), 't']] 3.562
 > Model[[(0, 1, 1), (0, 0, 1, 12), 'ct']] 4.703
 > Model[[(0, 1, 1), (0, 0, 2, 0), 'ct']] 4.707
 > Model[[(0, 1, 1), (0, 1, 0, 12), 'ct']] 2.416
 > Model[[(0, 1, 1), (1, 0, 0, 0), 'ct']] 5.015
 > Model[[(0, 1, 1), (1, 0, 1, 0), 'ct']] 5.005
 > Model[[(0, 1, 1), (1, 0, 0, 12), 'ct']] 2.982
 > Model[[(0, 1, 1), (1, 0, 2, 0), 'ct']] 4.979
 > Model[[(0, 1, 1), (2, 0, 0, 0), 'ct']] 4.902
 > Model[[(0, 1, 1), (1, 0, 1, 12), 'ct']] 2.644
 > Model[[(0, 1, 1), (1, 1, 0, 12), 'ct']] 2.201
 > Model[[(0, 1, 1), (2, 0, 1, 0), 'ct']] 4.907
 > Model[[(0, 1, 1), (2, 0, 0, 12), 'ct']] 2.443
 > Model[[(0, 1, 2), (0, 0, 0, 0), 'n']] 4.534
 > Model[[(0, 1, 2), (0, 0, 0, 12), 'n']] 4.534
 > Model[[(0, 1, 2), (0, 0, 1, 0), 'n']] 4.508
 > Model[[(0, 1, 1), (2, 0, 2, 0), 'ct']] 2.744
 > Model[[(0, 1, 1), (2, 0, 1, 12), 'ct']] 2.797
 > Model[[(0, 1, 2), (0, 0, 2, 0), 'n']] 4.441
 > Model[[(0, 1, 2), (0, 1, 0, 12), 

 > Model[[(1, 0, 0), (2, 1, 0, 12), 'ct']] 2.868
 > Model[[(1, 0, 1), (2, 0, 0, 0), 'n']] 4.452
 > Model[[(1, 0, 1), (2, 0, 1, 0), 'n']] 4.884
 > Model[[(1, 0, 1), (2, 0, 2, 0), 'n']] 4.243
 > Model[[(1, 0, 1), (2, 0, 0, 12), 'n']] 2.125
 > Model[[(1, 0, 1), (0, 0, 0, 0), 'c']] 4.743
 > Model[[(1, 0, 1), (0, 0, 0, 12), 'c']] 4.743
 > Model[[(1, 0, 1), (2, 1, 0, 12), 'n']] 3.692
 > Model[[(1, 0, 1), (2, 0, 1, 12), 'n']] 2.458
 > Model[[(1, 0, 1), (0, 1, 0, 12), 'c']] 2.354
 > Model[[(1, 0, 1), (0, 0, 1, 0), 'c']] 4.681
 > Model[[(1, 0, 1), (0, 0, 2, 0), 'c']] 3.857
 > Model[[(1, 0, 1), (0, 0, 1, 12), 'c']] 4.155
 > Model[[(1, 0, 1), (1, 0, 0, 0), 'c']] 4.464
 > Model[[(1, 0, 1), (1, 0, 1, 0), 'c']] 4.473
 > Model[[(1, 0, 1), (1, 0, 0, 12), 'c']] 2.970
 > Model[[(1, 0, 1), (1, 0, 2, 0), 'c']] 3.992
 > Model[[(1, 0, 1), (1, 0, 1, 12), 'c']] 3.116
 > Model[[(1, 0, 1), (1, 1, 0, 12), 'c']] 2.220
 > Model[[(1, 0, 1), (2, 0, 0, 0), 'c']] 3.445
 > Model[[(1, 0, 1), (2, 0, 1, 0), 'c']] 2.812
 >

 > Model[[(1, 1, 0), (0, 1, 0, 12), 't']] 3.317
 > Model[[(1, 1, 0), (0, 0, 1, 12), 't']] 4.703
 > Model[[(1, 1, 0), (0, 0, 2, 0), 't']] 4.631
 > Model[[(1, 1, 0), (1, 0, 0, 0), 't']] 4.883
 > Model[[(1, 1, 0), (1, 0, 0, 12), 't']] 3.530
 > Model[[(1, 1, 0), (1, 0, 1, 0), 't']] 5.004
 > Model[[(1, 1, 0), (2, 0, 0, 0), 't']] 4.748
 > Model[[(1, 1, 0), (1, 0, 1, 12), 't']] 2.603
 > Model[[(1, 1, 0), (1, 1, 0, 12), 't']] 2.786
 > Model[[(1, 1, 0), (1, 0, 2, 0), 't']] 5.118
 > Model[[(1, 1, 0), (2, 0, 0, 12), 't']] 2.903
 > Model[[(1, 1, 0), (2, 0, 1, 0), 't']] 4.782
 > Model[[(1, 1, 0), (2, 0, 2, 0), 't']] 3.693
 > Model[[(1, 1, 0), (0, 0, 0, 0), 'ct']] 5.049
 > Model[[(1, 1, 0), (0, 0, 0, 12), 'ct']] 5.049
 > Model[[(1, 1, 0), (0, 0, 1, 0), 'ct']] 5.014
 > Model[[(1, 1, 0), (2, 0, 1, 12), 't']] 3.203
 > Model[[(1, 1, 0), (0, 1, 0, 12), 'ct']] 3.317
 > Model[[(1, 1, 0), (0, 0, 2, 0), 'ct']] 4.689
 > Model[[(1, 1, 0), (0, 0, 1, 12), 'ct']] 4.806
 > Model[[(1, 1, 0), (1, 0, 0, 0), 'ct']] 4.

 > Model[[(1, 1, 2), (2, 0, 1, 0), 'ct']] 4.162
 > Model[[(1, 1, 2), (2, 0, 0, 12), 'ct']] 2.521
 > Model[[(2, 0, 0), (0, 0, 0, 0), 'n']] 4.895
 > Model[[(2, 0, 0), (0, 0, 0, 12), 'n']] 4.895
 > Model[[(1, 1, 2), (2, 0, 2, 0), 'ct']] 2.666
 > Model[[(2, 0, 0), (0, 0, 1, 0), 'n']] 4.935
 > Model[[(1, 1, 2), (2, 0, 1, 12), 'ct']] 2.799
 > Model[[(2, 0, 0), (0, 1, 0, 12), 'n']] 2.241
 > Model[[(2, 0, 0), (0, 0, 1, 12), 'n']] 4.610
 > Model[[(2, 0, 0), (1, 0, 0, 0), 'n']] 4.771
 > Model[[(2, 0, 0), (0, 0, 2, 0), 'n']] 4.507
 > Model[[(2, 0, 0), (1, 0, 0, 12), 'n']] 3.420
 > Model[[(2, 0, 0), (1, 0, 1, 0), 'n']] 4.531
 > Model[[(2, 0, 0), (1, 0, 1, 12), 'n']] 2.303
 > Model[[(2, 0, 0), (1, 1, 0, 12), 'n']] 1.820
 > Model[[(2, 0, 0), (1, 0, 2, 0), 'n']] 4.558
 > Model[[(2, 0, 0), (2, 0, 0, 0), 'n']] 4.797
 > Model[[(2, 0, 0), (2, 0, 0, 12), 'n']] 1.863
 > Model[[(2, 0, 0), (2, 0, 1, 0), 'n']] 3.224
 > Model[[(1, 1, 2), (2, 1, 0, 12), 'ct']] 4.279
 > Model[[(2, 0, 0), (0, 0, 0, 0), 'c']] 4.37

 > Model[[(2, 0, 2), (0, 1, 0, 12), 'c']] 2.954
 > Model[[(2, 0, 2), (1, 0, 0, 0), 'c']] 2.416
 > Model[[(2, 0, 2), (1, 0, 1, 0), 'c']] 2.255
 > Model[[(2, 0, 2), (1, 0, 0, 12), 'c']] 2.314
 > Model[[(2, 0, 2), (1, 0, 1, 12), 'c']] 8.007
 > Model[[(2, 0, 2), (1, 0, 2, 0), 'c']] 2.369
 > Model[[(2, 0, 2), (1, 1, 0, 12), 'c']] 2.162
 > Model[[(2, 0, 2), (2, 0, 0, 0), 'c']] 2.454
 > Model[[(2, 0, 2), (2, 0, 1, 0), 'c']] 2.904
 > Model[[(2, 0, 2), (2, 0, 2, 0), 'c']] 2.149
 > Model[[(2, 0, 2), (2, 0, 0, 12), 'c']] 2.070
 > Model[[(2, 0, 2), (0, 0, 0, 0), 't']] 4.542
 > Model[[(2, 0, 2), (0, 0, 0, 12), 't']] 4.542
 > Model[[(2, 0, 2), (0, 0, 1, 0), 't']] 4.350
 > Model[[(2, 0, 2), (2, 0, 1, 12), 'c']] 6.674
 > Model[[(2, 0, 2), (0, 0, 1, 12), 't']] 4.302
 > Model[[(2, 0, 2), (0, 0, 2, 0), 't']] 5.357
 > Model[[(2, 0, 2), (0, 1, 0, 12), 't']] 3.102
 > Model[[(2, 0, 2), (1, 0, 0, 0), 't']] 3.446
 > Model[[(2, 0, 2), (2, 1, 0, 12), 'c']] 3.621
 > Model[[(2, 0, 2), (1, 0, 0, 12), 't']] 2.737
 >

 > Model[[(2, 1, 1), (0, 0, 0, 0), 'ct']] 4.902
 > Model[[(2, 1, 1), (2, 0, 2, 0), 't']] 4.139
 > Model[[(2, 1, 1), (2, 0, 1, 12), 't']] 22335.837
 > Model[[(2, 1, 1), (0, 0, 0, 12), 'ct']] 4.902
 > Model[[(2, 1, 1), (0, 0, 1, 0), 'ct']] 4.909
 > Model[[(2, 1, 1), (0, 0, 1, 12), 'ct']] 161610.143
 > Model[[(2, 1, 1), (0, 0, 2, 0), 'ct']] 4.890
 > Model[[(2, 1, 1), (0, 1, 0, 12), 'ct']] 2.464
 > Model[[(2, 1, 1), (1, 0, 0, 0), 'ct']] 4.868
 > Model[[(2, 1, 1), (2, 1, 0, 12), 't']] 3.988
 > Model[[(2, 1, 1), (1, 0, 0, 12), 'ct']] 3.451
 > Model[[(2, 1, 1), (1, 0, 1, 0), 'ct']] 4.956
 > Model[[(2, 1, 1), (1, 0, 1, 12), 'ct']] 23923.970
 > Model[[(2, 1, 1), (2, 0, 0, 0), 'ct']] 4.888
 > Model[[(2, 1, 1), (1, 0, 2, 0), 'ct']] 4.784
 > Model[[(2, 1, 1), (1, 1, 0, 12), 'ct']] 2.253
 > Model[[(2, 1, 1), (2, 0, 1, 0), 'ct']] 4.826
 > Model[[(2, 1, 1), (2, 0, 0, 12), 'ct']] 2.897
 > Model[[(2, 1, 1), (2, 0, 2, 0), 'ct']] 3.551
 > Model[[(2, 1, 2), (0, 0, 0, 0), 'n']] 3.572
 > Model[[(2, 1, 1), (

We can see that the best result was an RMSE of about 1.52 degrees. A naive model achieved an RMSE of 1.50 degrees, suggesting that the best performing SARIMA model is not skillful on this problem. We can unpack the configuration of the best performing model as follows:

- Trend Order: (0, 0, 0)
- Seasonal Order: (1, 0, 1, 12)
- Trend Parameter: ‘n’ (no trend)

As we would expect, the model has no trend component and a 12-month seasonal ARIMA component.

## 13.6 Case Study 4: Trend and Seasonality

- The monthly car sales dataset summarizes the monthly car sales in Quebec, Canada between 1960 and 1968
- We will use the last year or 12 observations as the test set. The period of the seasonal component could be six months or 12 months. We will try both as the seasonal period in the call to the sarima configs() function when preparing the model configurations.

In [6]:
# grid search sarima hyperparameters for monthly car sales dataset
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_csv

# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

if __name__ == '__main__':
	# load dataset
	series = read_csv('monthly-car-sales.csv', header=0, index_col=0)
	data = series.values
	# data split
	n_test = 12
	# model configs
	cfg_list = sarima_configs(seasonal=[0,6,12])
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

 > Model[[(0, 0, 0), (0, 0, 0, 6), 'n']] 18608.652
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 18608.652
 > Model[[(0, 0, 0), (0, 0, 0, 12), 'n']] 18608.652
 > Model[[(0, 0, 0), (0, 0, 1, 6), 'n']] 13642.801
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'n']] 9928.438
 > Model[[(0, 0, 0), (0, 0, 1, 12), 'n']] 11897.520
 > Model[[(0, 0, 0), (0, 1, 0, 6), 'n']] 5798.153
 > Model[[(0, 0, 0), (0, 0, 2, 0), 'n']] 7292.275
 > Model[[(0, 0, 0), (0, 1, 0, 12), 'n']] 2290.827
 > Model[[(0, 0, 0), (0, 0, 2, 6), 'n']] 8191.126
 > Model[[(0, 0, 0), (0, 1, 1, 12), 'n']] 2286.289
 > Model[[(0, 0, 0), (0, 1, 1, 6), 'n']] 3958.642
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'n']] 3798.000
 > Model[[(0, 0, 0), (1, 0, 0, 6), 'n']] 5951.270
 > Model[[(0, 0, 0), (1, 0, 0, 12), 'n']] 1850.671
 > Model[[(0, 0, 0), (0, 1, 2, 6), 'n']] 3044.364
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'n']] 3942.265
 > Model[[(0, 0, 0), (1, 0, 1, 6), 'n']] 3764.214
 > Model[[(0, 0, 0), (0, 0, 2, 12), 'n']] 170976.744
 > Model[[(0, 0, 0), (1, 0, 1, 12), '

 > Model[[(0, 0, 0), (1, 1, 1, 12), 'ct']] 1975.630
 > Model[[(0, 0, 0), (1, 0, 2, 12), 'ct']] 9055.093
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'ct']] 3391.719
 > Model[[(0, 0, 0), (2, 0, 0, 6), 'ct']] 2463.848
 > Model[[(0, 0, 0), (2, 0, 1, 0), 'ct']] 3140.600
 > Model[[(0, 0, 0), (1, 1, 2, 6), 'ct']] 2392.498
 > Model[[(0, 0, 0), (2, 0, 0, 12), 'ct']] 3257.919
 > Model[[(0, 0, 0), (2, 0, 1, 6), 'ct']] 2162.069
 > Model[[(0, 0, 0), (2, 0, 2, 0), 'ct']] 2975.532
 > Model[[(0, 0, 0), (2, 0, 1, 12), 'ct']] 2894.969
 > Model[[(0, 0, 0), (2, 0, 2, 6), 'ct']] 2203.444
 > Model[[(0, 0, 0), (2, 1, 0, 6), 'ct']] 1645.557
 > Model[[(0, 0, 0), (2, 0, 2, 12), 'ct']] 7669.210
 > Model[[(0, 0, 0), (2, 1, 1, 6), 'ct']] 2398.245
 > Model[[(0, 0, 0), (2, 1, 0, 12), 'ct']] 1769.694
 > Model[[(0, 0, 0), (1, 1, 2, 12), 'ct']] 888401967.607
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'n']] 9928.438
 > Model[[(0, 0, 1), (0, 0, 0, 6), 'n']] 9928.438
 > Model[[(0, 0, 1), (0, 0, 0, 12), 'n']] 9928.438
 > Model[[(0, 0, 1), 

 > Model[[(0, 0, 1), (0, 1, 0, 12), 'ct']] 2020.524
 > Model[[(0, 0, 1), (0, 0, 2, 6), 'ct']] 2034.255
 > Model[[(0, 0, 1), (0, 1, 1, 6), 'ct']] 3348.444
 > Model[[(0, 0, 1), (0, 1, 1, 12), 'ct']] 1791.113
 > Model[[(0, 0, 1), (0, 0, 2, 12), 'ct']] 8758.673
 > Model[[(0, 0, 1), (0, 1, 2, 6), 'ct']] 2781.836
 > Model[[(0, 0, 1), (1, 0, 0, 0), 'ct']] 3396.552
 > Model[[(0, 0, 1), (1, 0, 0, 6), 'ct']] 3436.297
 > Model[[(0, 0, 1), (1, 0, 1, 0), 'ct']] 3426.626
 > Model[[(0, 0, 1), (1, 0, 0, 12), 'ct']] 2826.957
 > Model[[(0, 0, 1), (1, 0, 1, 6), 'ct']] 3412.130
 > Model[[(0, 0, 1), (2, 1, 2, 12), 't']] 172187252681.668
 > Model[[(0, 0, 1), (1, 0, 1, 12), 'ct']] 2226.236
 > Model[[(0, 0, 1), (1, 0, 2, 0), 'ct']] 3264.448
 > Model[[(0, 0, 1), (1, 1, 0, 6), 'ct']] 1797.830
 > Model[[(0, 0, 1), (1, 0, 2, 6), 'ct']] 2358.238
 > Model[[(0, 0, 1), (1, 1, 0, 12), 'ct']] 1845.030
 > Model[[(0, 0, 1), (0, 1, 2, 12), 'ct']] 15898315095.402
 > Model[[(0, 0, 1), (1, 1, 1, 6), 'ct']] 2000.343
 > Model[

 > Model[[(0, 0, 2), (2, 0, 1, 0), 't']] 3438.363
 > Model[[(0, 0, 2), (2, 0, 1, 6), 't']] 2293.204
 > Model[[(0, 0, 2), (2, 0, 2, 0), 't']] 3828.629
 > Model[[(0, 0, 2), (2, 0, 1, 12), 't']] 2337.259
 > Model[[(0, 0, 2), (2, 0, 2, 6), 't']] 2737.060
 > Model[[(0, 0, 2), (2, 1, 0, 6), 't']] 2171.867
 > Model[[(0, 0, 2), (1, 1, 2, 12), 't']] 38255772701.347
 > Model[[(0, 0, 2), (2, 1, 0, 12), 't']] 1887.829
 > Model[[(0, 0, 2), (2, 0, 2, 12), 't']] 4479.785
 > Model[[(0, 0, 2), (2, 1, 1, 6), 't']] 2887.007
 > Model[[(0, 0, 2), (0, 0, 0, 0), 'ct']] 3303.563
 > Model[[(0, 0, 2), (0, 0, 0, 6), 'ct']] 3303.563
 > Model[[(0, 0, 2), (0, 0, 0, 12), 'ct']] 3303.563
 > Model[[(0, 0, 2), (0, 0, 1, 0), 'ct']] 3023.318
 > Model[[(0, 0, 2), (2, 1, 2, 6), 't']] 3313.653
 > Model[[(0, 0, 2), (0, 0, 1, 6), 'ct']] 3287.631
 > Model[[(0, 0, 2), (0, 0, 1, 12), 'ct']] 2502.310
 > Model[[(0, 0, 2), (0, 0, 2, 0), 'ct']] 3145.196
 > Model[[(0, 0, 2), (2, 1, 1, 12), 't']] 2111.318
 > Model[[(0, 0, 2), (0, 0, 2

 > Model[[(0, 1, 0), (1, 0, 0, 0), 't']] 3966.858
 > Model[[(0, 1, 0), (0, 0, 2, 12), 't']] 2996.240
 > Model[[(0, 1, 0), (1, 0, 0, 6), 't']] 3886.881
 > Model[[(0, 1, 0), (1, 0, 0, 12), 't']] 2544.957
 > Model[[(0, 1, 0), (1, 0, 1, 0), 't']] 3449.467
 > Model[[(0, 1, 0), (1, 0, 1, 6), 't']] 3406.812
 > Model[[(0, 1, 0), (1, 0, 1, 12), 't']] 2451.711
 > Model[[(0, 1, 0), (1, 0, 2, 0), 't']] 3517.674
 > Model[[(0, 1, 0), (2, 1, 2, 12), 'c']] 955724657446.897
 > Model[[(0, 1, 0), (1, 0, 2, 6), 't']] 2549.014
 > Model[[(0, 1, 0), (1, 1, 0, 6), 't']] 2708.546
 > Model[[(0, 1, 0), (1, 1, 0, 12), 't']] 2722.177
 > Model[[(0, 1, 0), (1, 1, 1, 6), 't']] 2934.935
 > Model[[(0, 1, 0), (1, 0, 2, 12), 't']] 2653.440
 > Model[[(0, 1, 0), (1, 1, 1, 12), 't']] 2539.729
 > Model[[(0, 1, 0), (2, 0, 0, 0), 't']] 4049.121
 > Model[[(0, 1, 0), (0, 1, 2, 12), 't']] 5961599182.474
 > Model[[(0, 1, 0), (2, 0, 0, 6), 't']] 2579.179
 > Model[[(0, 1, 0), (1, 1, 2, 6), 't']] 2559.955
 > Model[[(0, 1, 0), (2, 0, 

...

 > Model[[(2, 0, 2), (2, 1, 1, 12), 'n']] 1755.759
 > Model[[(2, 0, 2), (0, 1, 0, 12), 'c']] 1925.090
 > Model[[(2, 0, 2), (0, 1, 0, 6), 'c']] 3025.635
 > Model[[(2, 0, 2), (0, 1, 1, 12), 'c']] 1680.213
 > Model[[(2, 0, 2), (0, 0, 2, 12), 'c']] 7254.649
 > Model[[(2, 0, 2), (0, 1, 1, 6), 'c']] 2052.540
 > Model[[(2, 0, 2), (1, 0, 0, 0), 'c']] 3146.132
 > Model[[(2, 0, 2), (1, 0, 0, 6), 'c']] 3500.037
 > Model[[(2, 0, 2), (2, 1, 2, 12), 'n']] 372920596079.259
 > Model[[(2, 0, 2), (0, 1, 2, 6), 'c']] 2689.505
 > Model[[(2, 0, 2), (1, 0, 0, 12), 'c']] 2292.626
 > Model[[(2, 0, 2), (1, 0, 1, 0), 'c']] 3087.961
 > Model[[(2, 0, 2), (1, 0, 1, 6), 'c']] 3092.080
 > Model[[(2, 0, 2), (1, 0, 2, 0), 'c']] 2706.417
 > Model[[(2, 0, 2), (1, 0, 2, 6), 'c']] 2529.143
 > Model[[(2, 0, 2), (1, 0, 1, 12), 'c']] 2045.381
 > Model[[(2, 0, 2), (1, 1, 0, 12), 'c']] 1741.983
 > Model[[(2, 0, 2), (0, 1, 2, 12), 'c']] 156680521453.866
 > Model[[(2, 0, 2), (1, 1, 0, 6), 'c']] 2290.560
 > Model[[(2, 0, 2), (1, 

 > Model[[(2, 1, 0), (2, 0, 1, 0), 'n']] 3319.671
 > Model[[(2, 1, 0), (1, 1, 2, 6), 'n']] 1988.418
 > Model[[(2, 1, 0), (2, 0, 1, 6), 'n']] 2133.521
 > Model[[(2, 1, 0), (2, 0, 2, 0), 'n']] 2991.855
 > Model[[(2, 1, 0), (2, 0, 1, 12), 'n']] 1963.251
 > Model[[(2, 1, 0), (2, 0, 2, 6), 'n']] 2063.669
 > Model[[(2, 1, 0), (2, 1, 0, 6), 'n']] 2126.919
 > Model[[(2, 1, 0), (2, 0, 2, 12), 'n']] 2225.182
 > Model[[(2, 1, 0), (2, 1, 0, 12), 'n']] 1847.342
 > Model[[(2, 1, 0), (1, 1, 2, 12), 'n']] 16239251853.456
 > Model[[(2, 1, 0), (2, 1, 1, 6), 'n']] 1973.656
 > Model[[(2, 1, 0), (0, 0, 0, 0), 'c']] 4007.617
 > Model[[(2, 1, 0), (0, 0, 0, 6), 'c']] 4007.617
 > Model[[(2, 1, 0), (0, 0, 0, 12), 'c']] 4007.617
 > Model[[(2, 1, 0), (2, 1, 2, 6), 'n']] 2190.721
 > Model[[(2, 1, 0), (0, 0, 1, 0), 'c']] 3413.391
 > Model[[(2, 1, 0), (0, 0, 1, 6), 'c']] 4002.260
 > Model[[(2, 1, 0), (0, 0, 1, 12), 'c']] 2451.763
 > Model[[(2, 1, 0), (2, 1, 1, 12), 'n']] 2222.357
 > Model[[(2, 1, 0), (0, 0, 2, 0), '

 > Model[[(2, 1, 1), (1, 0, 0, 0), 'n']] 3344.786
 > Model[[(2, 1, 1), (1, 0, 0, 6), 'n']] 3099.992
 > Model[[(2, 1, 1), (0, 1, 2, 6), 'n']] 3172.337
 > Model[[(2, 1, 1), (1, 0, 1, 0), 'n']] 3275.362
 > Model[[(2, 1, 1), (1, 0, 0, 12), 'n']] 1891.686
 > Model[[(2, 1, 0), (2, 1, 2, 12), 'ct']] 314023357494.923
 > Model[[(2, 1, 1), (0, 1, 2, 12), 'n']] 63437531564.521
 > Model[[(2, 1, 1), (1, 0, 1, 6), 'n']] 3065.859
 > Model[[(2, 1, 1), (1, 0, 2, 0), 'n']] 3104.444
 > Model[[(2, 1, 1), (1, 0, 1, 12), 'n']] 1768.462
 > Model[[(2, 1, 1), (1, 0, 2, 6), 'n']] 2636.423
 > Model[[(2, 1, 1), (1, 1, 0, 6), 'n']] 1986.907
 > Model[[(2, 1, 1), (1, 1, 0, 12), 'n']] 1985.324
 > Model[[(2, 1, 1), (1, 0, 2, 12), 'n']] 1948.663
 > Model[[(2, 1, 1), (1, 1, 1, 6), 'n']] 2133.651
 > Model[[(2, 1, 1), (2, 0, 0, 0), 'n']] 3324.116
 > Model[[(2, 1, 1), (1, 1, 1, 12), 'n']] 1803.188
 > Model[[(2, 1, 1), (1, 1, 2, 6), 'n']] 1996.591
 > Model[[(2, 1, 1), (2, 0, 0, 6), 'n']] 2214.813
 > Model[[(2, 1, 1), (1, 1,

 > Model[[(2, 1, 1), (2, 0, 2, 6), 'ct']] 1990.518
 > Model[[(2, 1, 1), (2, 1, 0, 6), 'ct']] 2925.616
 > Model[[(2, 1, 1), (2, 0, 2, 12), 'ct']] 2645.098
 > Model[[(2, 1, 1), (2, 1, 1, 6), 'ct']] 2071.508
 > Model[[(2, 1, 1), (2, 1, 2, 6), 'ct']] 2171.198
 > Model[[(2, 1, 2), (0, 0, 0, 0), 'n']] 3221.606
 > Model[[(2, 1, 2), (0, 0, 0, 6), 'n']] 3221.606
 > Model[[(2, 1, 1), (2, 1, 0, 12), 'ct']] 2038.006
 > Model[[(2, 1, 2), (0, 0, 0, 12), 'n']] 3221.606
 > Model[[(2, 1, 1), (2, 1, 2, 12), 'ct']] 1988493357966.980
 > Model[[(2, 1, 2), (0, 0, 1, 0), 'n']] 2962.559
 > Model[[(2, 1, 2), (0, 0, 1, 6), 'n']] 2580.644
 > Model[[(2, 1, 2), (0, 0, 1, 12), 'n']] 2390.427
 > Model[[(2, 1, 2), (0, 0, 2, 0), 'n']] 2976.246
 > Model[[(2, 1, 2), (0, 1, 0, 6), 'n']] 4509.491
 > Model[[(2, 1, 2), (0, 0, 2, 6), 'n']] 2494.596
 > Model[[(2, 1, 2), (0, 1, 0, 12), 'n']] 2127.991
 > Model[[(2, 1, 2), (0, 1, 1, 6), 'n']] 2972.417
 > Model[[(2, 1, 1), (2, 1, 1, 12), 'ct']] 2357.525
 > Model[[(2, 1, 2), (0, 0

 > Model[[(2, 1, 2), (1, 0, 1, 6), 'ct']] 1874.765
 > Model[[(2, 1, 2), (1, 0, 1, 12), 'ct']] 2318.389
 > Model[[(2, 1, 2), (1, 0, 2, 0), 'ct']] 3019.854
 > Model[[(2, 1, 2), (0, 1, 2, 12), 'ct']] 597926755600.487
 > Model[[(2, 1, 2), (1, 0, 2, 6), 'ct']] 2085.492
 > Model[[(2, 1, 2), (1, 1, 0, 6), 'ct']] 3096.086
 > Model[[(2, 1, 2), (1, 1, 0, 12), 'ct']] 2285.473
 > Model[[(2, 1, 2), (1, 1, 1, 6), 'ct']] 2882.337
 > Model[[(2, 1, 2), (1, 0, 2, 12), 'ct']] 2455.786
 > Model[[(2, 1, 2), (2, 0, 0, 0), 'ct']] 3045.320
 > Model[[(2, 1, 2), (1, 1, 1, 12), 'ct']] 2064.704
 > Model[[(2, 1, 2), (1, 1, 2, 6), 'ct']] 2117.722
 > Model[[(2, 1, 2), (2, 0, 0, 6), 'ct']] 2360.657
 > Model[[(2, 1, 2), (2, 0, 1, 0), 'ct']] 3341.904
 > Model[[(2, 1, 2), (1, 1, 2, 12), 'ct']] 22016514598.876
 > Model[[(2, 1, 2), (2, 0, 0, 12), 'ct']] 2336.183
 > Model[[(2, 1, 2), (2, 0, 1, 6), 'ct']] 2309.312
 > Model[[(2, 1, 2), (2, 0, 2, 0), 'ct']] 2617.898
 > Model[[(2, 1, 2), (2, 0, 2, 6), 'ct']] 1966.422
 > Model[

We can see that the best result was an RMSE of about 1,551.84 sales. A naive model achieved an RMSE of 1,841.15 sales on this problem, suggesting that the best performing SARIMA model is skillful. We can unpack the configuration of the best performing model as follows:
- Trend Order: (0, 0, 0)
- Seasonal Order: (1, 1, 0, 12)
- Trend Parameter: ‘t’ (linear trend)

## 13.8 Further Reading

### 13.8.1 Books
- Chapter 8 ARIMA models, Forecasting: principles and practice, 2013. https://amzn.to/2xlJsfV
- Chapter 7, Non-stationary Models, Introductory Time Series with R, 2009. https://amzn.to/2smB9LR

## 13.8.2 APIs
- Statsmodels Time Series Analysis by State Space Methods. http://www.statsmodels.org/dev/statespace.html
- statsmodels.tsa.statespace.sarimax.SARIMAX API. http://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html
- statsmodels.tsa.statespace.sarimax.SARIMAXResults API. http://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAXResults.html
- Statsmodels SARIMAX Notebook. http://www.statsmodels.org/dev/examples/notebooks/generated/statespace_sarimax_stata.html

## 13.8.3 Articles
- Autoregressive integrated moving average, Wikipedia. https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average